In [12]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np
import pickle

import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from keras import models
from keras.models import Sequential
from keras.optimizers import Adam
import tensorflow as tf
import os
import copy

In [13]:
learning_rate = 0.001
weight_decay = 0.0001
batch_size = 128
num_epochs = 5
image_size = 72
num_heads = 4
projection_dim = 64
transformer_units = [
    projection_dim * 2,
    projection_dim
]
transformer_layers = 8
mlp_head_units = [2048, 1048] 

num_classes = 10
input_shape=(32,32,3)

In [14]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation = tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

In [15]:
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images = images,
            sizes = [1, self.patch_size, self.patch_size, 1],
            strides = [1, self.patch_size, self.patch_size, 1],
            rates = [1, 1, 1, 1],
            padding = "VALID", 
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

In [16]:
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units = projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim = num_patches, output_dim = projection_dim
        )
    def call(self, patch):
        positions = tf.range(start = 0, limit = self.num_patches, delta = 1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded
        

In [17]:
def create_vit_classifier(input_patch_size, input_data_augmentation):
    inputs = layers.Input(shape=input_shape)
    #augmented data
    augmented = input_data_augmentation(inputs)
    #create patches
    patches = Patches(input_patch_size)(augmented)
    # encode patches
    num_patches = (image_size // input_patch_size) ** 2
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # Create multiple layers of the Transformer Block
    for _ in range(transformer_layers):
        # layer normalisation
        x1 = layers.LayerNormalization(epsilon= 1e-6)(encoded_patches)
        # Create a multi -  head attention layer
        attention_output = layers.MultiHeadAttention(
            num_heads = num_heads,
            key_dim = projection_dim,
            dropout = 0
        )(x1, x1)
        # Skip connection 1
        x2 = layers.Add()([attention_output, encoded_patches])
        #layer normalisation
        x3 = layers.LayerNormalization(epsilon = 1e-6)(x2)
        # MLP
        x2 = mlp(x3, hidden_units = transformer_units, dropout_rate = 0)
        # skip connection2
        encoded_patches =  layers.Add()([x3, x2])
    # Create a [batch_size, projection_dim] tensor
    representation = layers.LayerNormalization(epsilon = 1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0)(representation)

    # Add MLP 
    features = mlp(representation, hidden_units = mlp_head_units, dropout_rate = 0)
    #classify ouputs
    logits = layers.Dense(num_classes)(features)
    # Create the Keras Model
    model = keras.Model(inputs = inputs, outputs = logits)
    return model

In [18]:
def build_ViT_model(input_patch_size, input_data_augmentation):
    
    model = create_vit_classifier(input_patch_size, input_data_augmentation)
    optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )
    model.compile(
        optimizer=optimizer,
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[
            keras.metrics.SparseCategoricalAccuracy(name="accuracy")
        ]
    )

    return model

In [19]:
def get_processed_dataset(dir):
    train_npy = np.load(dir)[:6000]
    label_npy = np.load('train_labels.npy')

    return train_npy, label_npy

In [20]:
patch_size_configs = [12, 18, 24]

train_accuracy_list = []
test_accuracy_list  = []
test_precision_list  = []
test_recall_list  = []

In [21]:
ORIGIN = 'origin_cifar10.npy'
SMALL = 'small_noise_cifar10.npy'
MEDIUM = 'medium_noise_cifar10.npy'
LARGE = 'large_noise_cifar10.npy'

In [22]:
outer_cv = KFold(n_splits=3, shuffle=True, random_state=42)
best_params = {}
performance_results = []

x_combined, y_combined = get_processed_dataset(LARGE)

data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        layers.Resizing(image_size, image_size)
    ],
    name = "data_augmentation"
)
data_augmentation.layers[0].adapt(x_combined)


index_k_1 = 0

for outer_train_idx, outer_val_idx in outer_cv.split(x_combined):
    # print("outer_val_idx:", outer_val_idx)
    # print(type(outer_val_idx))
    index_k_1 = index_k_1 + 1
    print("index_k_1", index_k_1)
    
    X_outer_train, X_outer_val = x_combined[outer_train_idx], x_combined[outer_val_idx]
    y_outer_train, y_outer_val = y_combined[outer_train_idx], y_combined[outer_val_idx]

    best_score = -np.inf
    best_patch_size = None

    for patch_size in patch_size_configs:
        print("patch_size:", patch_size)
        inner_cv = KFold(n_splits=10, shuffle=True, random_state=42)
        scores = []
        
        for inner_train_idx, inner_val_idx in inner_cv.split(X_outer_train):
            X_inner_train, X_inner_val = X_outer_train[inner_train_idx], X_outer_train[inner_val_idx]
            y_inner_train, y_inner_val = y_outer_train[inner_train_idx], y_outer_train[inner_val_idx]
            
            model = build_ViT_model(patch_size, data_augmentation)
            history = model.fit(
                x=X_inner_train,
                y=y_inner_train,
                batch_size=batch_size,
                epochs=num_epochs,
                validation_data=(X_inner_val, y_inner_val)
            )
            
            score = model.evaluate(X_inner_val, y_inner_val, verbose=0)[1] # accuracy
            scores.append(score)

        mean_score = np.mean(scores)
        print("mean_score:", mean_score, "( patch_size", patch_size, ")")
        
        if mean_score > best_score:
            best_score = mean_score
            best_patch_size = patch_size

    best_params[index_k_1] = best_patch_size
    final_model = build_ViT_model(best_patch_size, data_augmentation)
    final_model.fit(X_outer_train, y_outer_train, epochs=5, batch_size=128, verbose=0)
    final_performance = final_model.evaluate(X_outer_val, y_outer_val)[1]
    performance_results.append(final_performance)

index_k_1 1
patch_size: 12
Epoch 1/5
29/29 [==============================] - 19s 478ms/step - loss: 2.6430 - accuracy: 0.2289 - val_loss: 1.8544 - val_accuracy: 0.3550
Epoch 2/5
29/29 [==============================] - 13s 454ms/step - loss: 1.7406 - accuracy: 0.3728 - val_loss: 1.7506 - val_accuracy: 0.3625
Epoch 3/5
29/29 [==============================] - 13s 445ms/step - loss: 1.4746 - accuracy: 0.4714 - val_loss: 1.7558 - val_accuracy: 0.3675
Epoch 4/5
29/29 [==============================] - 13s 434ms/step - loss: 1.1647 - accuracy: 0.5917 - val_loss: 1.7386 - val_accuracy: 0.4275
Epoch 5/5
29/29 [==============================] - 13s 432ms/step - loss: 0.8531 - accuracy: 0.7147 - val_loss: 1.8872 - val_accuracy: 0.4075
Epoch 1/5
29/29 [==============================] - 18s 466ms/step - loss: 2.4261 - accuracy: 0.2603 - val_loss: 2.0378 - val_accuracy: 0.2750
Epoch 2/5
29/29 [==============================] - 13s 442ms/step - loss: 1.6478 - accuracy: 0.4217 - val_loss: 1.8739 - 

In [23]:
average_performance = np.mean(performance_results)
print(f'Average Performance across all outer folds: {average_performance}')
print(performance_results)
print(f'Best Parameters for each fold: {best_params}')

Average Performance across all outer folds: 0.4274999996026357
[0.43149998784065247, 0.42800000309944153, 0.4230000078678131]
Best Parameters for each fold: {1: 12, 2: 12, 3: 12}
